In [231]:
import numpy as np
from PIL import Image
import glob
from sklearn.datasets import fetch_olivetti_faces

In [232]:
# Define constants
IMG_SHAPE = (64, 64)
# Download Olivetti faces dataset
olivetti = fetch_olivetti_faces()
x = olivetti.images
y = olivetti.target
print(y)

# Print info on shapes and reshape where necessary
print("Original x shape:", x.shape)
X = x.reshape((400, 4096))
X_test = X[:200]
print(X_test.shape)
Y_test = X[201:300]


[ 0  0  0  0  0  0  0  0  0  0  1  1  1  1  1  1  1  1  1  1  2  2  2  2
  2  2  2  2  2  2  3  3  3  3  3  3  3  3  3  3  4  4  4  4  4  4  4  4
  4  4  5  5  5  5  5  5  5  5  5  5  6  6  6  6  6  6  6  6  6  6  7  7
  7  7  7  7  7  7  7  7  8  8  8  8  8  8  8  8  8  8  9  9  9  9  9  9
  9  9  9  9 10 10 10 10 10 10 10 10 10 10 11 11 11 11 11 11 11 11 11 11
 12 12 12 12 12 12 12 12 12 12 13 13 13 13 13 13 13 13 13 13 14 14 14 14
 14 14 14 14 14 14 15 15 15 15 15 15 15 15 15 15 16 16 16 16 16 16 16 16
 16 16 17 17 17 17 17 17 17 17 17 17 18 18 18 18 18 18 18 18 18 18 19 19
 19 19 19 19 19 19 19 19 20 20 20 20 20 20 20 20 20 20 21 21 21 21 21 21
 21 21 21 21 22 22 22 22 22 22 22 22 22 22 23 23 23 23 23 23 23 23 23 23
 24 24 24 24 24 24 24 24 24 24 25 25 25 25 25 25 25 25 25 25 26 26 26 26
 26 26 26 26 26 26 27 27 27 27 27 27 27 27 27 27 28 28 28 28 28 28 28 28
 28 28 29 29 29 29 29 29 29 29 29 29 30 30 30 30 30 30 30 30 30 30 31 31
 31 31 31 31 31 31 31 31 32 32 32 32 32 32 32 32 32

In [233]:
def crop_image(img):
    width, height = img.size

    new_width = IMG_SHAPE[0]
    new_height = IMG_SHAPE[1]
    left = (width - new_width)/2 - .5
    top = (height - new_height)/2 - .5
    right = (width + new_width)/2 - .5
    bottom = (height + new_height)/2 - .5

    # Crop the center of the image
    im = img.crop((left, top, right, bottom))

    return im

def img_to_vector(path_to_img):
    # Load the image
    img = Image.open(path_to_img).convert('L')
    img = crop_image(img)

    # Convert the image into n²*1 array
    arr = np.array(img)
    flat_array = arr.ravel()

    return flat_array


def vector_to_img(vector, shape = IMG_SHAPE):
    array = vector.reshape(shape)
    img = Image.fromarray(array)
    return img

def sum_of_vectors(arr: []):
    sum_vector = np.zeros(len(arr[0]))
    for i in range(0, len(arr[0])):
        for v in arr:
            sum_vector[i] += v[i]

    return sum_vector


def scalar_multiply_vector(scalar, v):
    arr = np.array([])
    for x in v:
        arr = np.append(arr, x * scalar)
    return arr


def negative_vector(v):
    new_v = []
    for x in v:
        new_v.append(-x)
    return np.array(new_v)

In [234]:
all_imgs = []
training_set_photo_names = []

# Get images from dataset and convert them to vectors
#for folder in glob.iglob('../dataset/*'):
#    for img in glob.iglob(folder + '/*'):
#        training_set_photo_names.append(img[img.rfind('/')+1:len(img)])
#        all_imgs.append(img_to_vector(img))


# Create an np.array from the vectors
training_set = X#np.array(all_imgs)

In [235]:
# Average face using numpy
avg_face = training_set.mean(axis=0)

In [236]:
# Let's create the matrix A by subtracting the average face from each face in the training set
A = []
neg_avg_face = negative_vector(avg_face)
sub = None
for v in training_set:
    sub = np.subtract(v, avg_face)
    A.append(sub)

# Convert A to a matrix
A_m = np.asmatrix(A)

A_t = np.array(A).transpose()

In [237]:
# Form the covariance matrix
cov_matrix = np.cov(np.array(A))

In [238]:
# Calculate the eigenvectors of the covariance matrix
eigenvalues, eigenvectors = np.linalg.eig(cov_matrix)
eig_pairs = [(eigenvalues[index], eigenvectors[:,index]) for index in range(len(eigenvalues))]

eig_pairs.sort(reverse=True)
eigvalues_sort  = [eig_pairs[index][0] for index in range(len(eigenvalues))]
eigvectors_sort = [eig_pairs[index][1] for index in range(len(eigenvalues))]

In [239]:
# Choose the 10 eigenvectors with the highest eigenvalues as the eigenfaces
eigenfaces = np.array(eigvectors_sort[:20]).transpose()

In [240]:
# Calculate eiganfaces
# Create reduced eigenface space
proj_data = np.dot(training_set.transpose(), eigenfaces)
proj_data = proj_data.transpose()
# Calculate weights for eigenfaces
w = np.array([np.dot(proj_data,i) for i in np.array(A)])

In [241]:
# Get images from dataset and convert them to vectors
unknown_face_vector        = Y_test[10]#img_to_vector('../test/Hugo_Chavez_0029.jpg')
normalized_u = np.subtract(unknown_face_vector, avg_face)
w_unknown = np.dot(proj_data, normalized_u)
difference_vector = w - w_unknown
norms = np.linalg.norm(difference_vector, axis=1)
index = np.argmin(norms)
print(index)
#training_set_photo_names[index]

211
